In [21]:
# This program is designed to provide data a heat map for S1 from a .csv file.

# This cell is composed of the necessary imports and class formations for the program.

# Used for math opperations:
import numpy as np
import math

# Used for plotting data:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from typing import Optional as Opt

# Used to read files:
import pandas as pd

In [2]:
'''This cell creates theta values that will be used to parametrically create a scatter plot of our trials,
which we can then approximate by curve fitting. getTheta takes in a positive integer 'n' and returns 
a list 'thetaVals' of theta values on the circle.'''

def getTheta(n: int) -> list[float]:
    thetaVals = [(2*np.pi*k)/n for k in range(n+1)]
    return thetaVals

In [10]:
# This cell creates the heat map from a .csv file of the form '10-30000_s0.0-0.5.csv' with createHeatMap3D.
# createHeatMap3D takes the .csv file name and a list 'sVales' of s values that the file includes.

def createHeatMap3D(file: str, sVals: list[float] = None) -> None:
    colnames = ['angle', 'trialDatum'] # Names the columns so they can be referrenced.
    df = pd.read_csv(file, names = colnames) # Reads the file and saves it to df with the above column names.
    angle, trialDatum = [df.angle, df.trialDatum] # Collects the column data in a list.
    allAngles = [] # Initiates a list to contain lists of angles as indexed by integers 0 to some n.
    angles = [] # Initates a list of angles is indexed by integers 0 to some n.
    allTrialData = [] # Initiates a list of lists of trial data associated to pairs (n, s).
    trialData = [] # Initiates a list of trial data.
    # Divides the .csv file into sections (lists) for each n by initiating a new list when the angle returns to 0:
    for i in range(len(angle)):
        angles.append(angle[i])
        trialData.append(trialDatum[i])
        if angle[i] == 0 and len(angles) > 1: # The length must be > 1 so that the first section is not [0].
            allAngles.append(angles) # Appends the completed section angles to 'allAngles'.
            allTrialData.append(trialData) # Appends the completed section trial data to 'allAngles'.
            # Ensures the each section begins and ends with 0 to form a circle:
            angles = [angle[0]]
            trialData = [trialDatum[0]] 
    # The final section will not be closed by a 0, so we have to do it manually.
    angles.append(angle[0])
    trialData.append(trialDatum[0])
    allAngles.append(angles)
    allTrialData.append(trialData)
    nVals = [len(angles) - 1 for angles in allAngles] # We subtract 1 to counter the final addition of an extra 0.
    allThetas = [getTheta(n) for n in nVals] # Creates a list of lists of thetas generated by getTheta.
    lines = [[0 for i in range(n + 1)] for n in nVals] # Lines will ensure that a circle lies on z = 0.
    plt.figure(figsize=(30,10*math.ceil(len(nVals)/3))) # Creates a figure of 30 square inches.
    # Creates a list of axes for 3d graphs:     
    axes = [plt.subplot(math.ceil(len(nVals)/3), 3, 1 + i, projection='3d') for i in range(len(nVals))]
    # Creates the 3d graphs:
    sCoord = 0 # Keeps track of the s value coordinate in sVals.
    for i in range(len(nVals)):
        # Plots a 3D height graph:
        axes[i].plot3D(np.cos(allThetas[i]), np.sin(allThetas[i]), allTrialData[i], alpha=0.25, c='maroon')
        # Plots a black circle with the same x- and y-values, but withz-values constantly 0.
        axes[i].plot3D(np.cos(allThetas[i]), np.sin(allThetas[i]), lines[i], alpha=.75, c='black')
        # Plots a heat map with the 'coolwarm' colormap on the black circle according to the values of allTrialData[i].
        axes[i].scatter3D(np.cos(allThetas[i]), np.sin(allThetas[i]), lines[i], cmap='coolwarm', c=allTrialData[i], alpha=1, s=100)
        # Labels the graph according to the n and s values..
        axes[i].set_title("n value: {0}, s value: {1}".format(nVals[i], sVals[sCoord]))
        # readjusts the s coordinate:
        if sCoord + 1 < len(sVals):
            sCoord += 1
        else:
            sCoord = 0
    plt.show() # Displays the graph.
    
    

# createHeatMap('10-30000_s0.0-0.5.csv', [0.000000, 0.225000, 0.250000, 0.275000, 0.500000])


In [22]:
'''This cell creates a 2D graph, whose x-axis is n values, and whose y-axis is E[max^n_s], given a 
.csv file of the form '500-900_s0.0-0.5.csv' and a specified parameter 's'. One can also specify the y-range
with ymin and ymax.'''

def createHeatMap2D(file: str, s: float, ymin: Opt[float] = 0, ymax: Opt[float] = 2) -> None:
    colnames = ['n', 'sValue', 'expectedMean'] # Names the columns so it can be referrenced.
    df = pd.read_csv(file, names = colnames) # Reads the file and saves it to df with the above column names.
    n, sValue, expectedMean  = [df.n, df.sValue, df.expectedMean] # Collects the column data in a list.
    nVals = [] # For the x-axis.
    mVals = [] # For the y-axis
    for i in range(len(df)):
        if sValue[i] == s: # Only looks at rows with s-value of the kind specified.
            nVals.append(n[i])
            mVals.append(expectedMean[i])
    f = plt.figure(figsize = (8,8)) # Sets the plot size.
    plt.scatter(x=nVals,y=mVals,s=50) # Creates the scatter plot.
    plt.ylim(ymin,ymax)

# createHeatMap2D('500-900_s0.0-0.5.csv', 0.5)
    